In [ ]:
import os
import tensorflow as tf
import numpy as np
from google.colab import drive
# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED=1234
tf.random.set_seed(SEED)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#cwd='/content/drive/My Drive/Kaggle1'
cwd='/content/drive/My Drive/AN2DL'

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import preprocess_input 


apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        preprocessing_function=preprocess_input,
                                        validation_split=0.2)
else:
    train_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                        validation_split=0.2) # set validation split

#the validation data generator
valid_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,validation_split=0.2)


In [ ]:
#dataset_dir='/content/drive/My Drive/Kaggle1/MaskDataset'
dataset_dir='/content/drive/My Drive/AN2DL/MaskDataset'
train_dir=os.path.join(dataset_dir,'training')
valid_dir=os.path.join(dataset_dir,'training')
test_dir=os.path.join(dataset_dir,'test')

bs=10

# img shape
img_h = 512
img_w = 512
train_generator = train_data_gen.flow_from_directory(
    train_dir,
    target_size=(img_h,img_w),
    batch_size=bs,
    class_mode='categorical', # targets are directly converted into one-hot vectors
    shuffle=True,seed=SEED,
    subset='training') # set as training data

validation_generator = train_data_gen.flow_from_directory(
    valid_dir, # same directory as training data
    target_size=(img_h,img_w),
    batch_size=bs,
    class_mode='categorical',
    subset='validation') # set as validation data


#train_generator.class_indices
num_classes=len(train_generator.class_indices)

train_dataset=tf.data.Dataset.from_generator(lambda:train_generator,output_types=(tf.float32,tf.float32),output_shapes=([None,img_h,img_w,3],[None,num_classes]))
train_dataset = train_dataset.repeat()


valid_dataset=tf.data.Dataset.from_generator(lambda:validation_generator,output_types=(tf.float32,tf.float32),output_shapes=([None,img_h,img_w,3],[None,num_classes]))
valid_dataset=valid_dataset.repeat()
iter(train_dataset)


Found 4492 images belonging to 3 classes.
Found 1122 images belonging to 3 classes.


In [ ]:
#check
import matplotlib.pyplot as plt
%matplotlib inline

iterator=iter(train_dataset)
augmented_img,target=next(iterator)
augmented_img=np.array(augmented_img[0])*255 # denormalize

#plt.imshow(np.uint8(augmented_img))
#plt.show()
iterator

In [ ]:
#import pretrained model without FC part
xception = tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

83689472/83683744 [==============================] - 0s 0us/step


In [ ]:
# Create Model
# ------------

finetuning = True

if finetuning:
    freeze_until = 40 # layer from which we want to fine-tune
    
    for layer in xception.layers[:freeze_until]:
        layer.trainable = False
else:
    xception.trainable = False
    
model = tf.keras.Sequential()
model.add(xception)
model.add(tf.keras.layers.GlobalMaxPooling2D(data_format=None))
model.add(tf.keras.layers.Dense(units=32, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Visualize created model as a table
model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 25, 25, 2048)      20861480  
_________________________________________________________________
global_max_pooling2d (Global (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                65568     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 99        
Total params: 20,927,147
Trainable params: 19,227,867
Non-trainable params: 1,699,280
_________________________________________________________________


In [ ]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,restore_best_weights=True)
    callbacks.append(es_callback)

# Learning rate adapter 
# --------------
lr_adapter = True
if lr_adapter:
    lr_adapter_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, mode='auto', min_delta=0.0001, cooldown=0,restore_best_weights=True)
    callbacks.append(lr_adapter_callback)

In [ ]:
#model training
model.fit(x=train_dataset,
          epochs=50, 
          steps_per_epoch=len(train_generator)/2,
          validation_data=valid_dataset,
          validation_steps=len(validation_generator), 
          callbacks=callbacks)

Epoch 1/50
225/225 [==============================] - 1574s 7s/step - loss: 0.5854 - accuracy: 0.7253 - val_loss: 0.2909 - val_accuracy: 0.8779
Epoch 2/50
225/225 [==============================] - 1216s 5s/step - loss: 0.2723 - accuracy: 0.8898 - val_loss: 0.2840 - val_accuracy: 0.8806
Epoch 3/50
225/225 [==============================] - 593s 3s/step - loss: 0.2033 - accuracy: 0.9267 - val_loss: 0.1981 - val_accuracy: 0.9225
Epoch 4/50
225/225 [==============================] - 592s 3s/step - loss: 0.1670 - accuracy: 0.9384 - val_loss: 0.2461 - val_accuracy: 0.9251
Epoch 5/50
225/225 [==============================] - 605s 3s/step - loss: 0.1509 - accuracy: 0.9467 - val_loss: 0.1351 - val_accuracy: 0.9519
Epoch 6/50
225/225 [==============================] - 626s 3s/step - loss: 0.1261 - accuracy: 0.9599 - val_loss: 0.1812 - val_accuracy: 0.9340
Epoch 7/50
225/225 [==============================] - 628s 3s/step - loss: 0.1081 - accuracy: 0.9649 - val_loss: 0.1444 - val_accuracy: 0.95

In [ ]:
model.save('/content/drive/My Drive/Kaggle1/TransferSavedModel/xception_final')


In [ ]:
#evaluate optimal weights on validation set
val=model.evaluate(valid_dataset,steps=len(validation_generator),verbose=1)

113/113 [==============================] - 121s 1s/step - loss: 0.1850 - accuracy: 0.9501


In [ ]:
#compute predictions
from PIL import Image

final_path = os.path.join(dataset_dir, 'test')
image_filenames = next(os.walk(final_path))[2] 

results={} 
for image_name in image_filenames:
  
   img=Image.open(final_path+'/'+image_name).convert('RGB')
   img=img.resize((img_h,img_w))
   img_array=np.array(img)
   img_array=preprocess_input(img_array)
   img_array=np.expand_dims(img_array, 0)
   pred=model.predict(img_array)
   prediction=np.argmax(pred)
   results[image_name]=prediction

In [ ]:
def create_csv(results, results_dir='./'):

    csv_fname = 'xception'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value)+ '\n')
create_csv(results,cwd)